In [1]:
import random
import numpy as np
import math


def generate_normal_random_numbers(count, mean, std_dev):
    """
    使用 numpy 生成一定数量 0-1 之间符合正态分布的随机数，并返回数组
    :param count: 生成的随机数数量
    :param mean: 正态分布的均值
    :param std_dev: 正态分布的标准差
    :return: 生成的随机数数组
    """
    random_numbers = np.empty(count)
    generated_count = 0

    while generated_count < count:
        remaining_count = count - generated_count
        new_numbers = np.random.normal(loc=mean, scale=std_dev, size=remaining_count)
        valid_numbers = new_numbers[(new_numbers >= 0) & (new_numbers <= 1)]

        valid_count = len(valid_numbers)
        random_numbers[generated_count : generated_count + valid_count] = valid_numbers
        generated_count += valid_count

    return random_numbers


def probability_majority_one(probabilities):
    n = len(probabilities)
    half = math.ceil(n / 2)
    if n % 2 == 0:
        half += 1

    # Initialize dp table
    dp = [[0] * (n + 1) for _ in range(n + 1)]
    dp[0][0] = 1

    # Fill dp table
    for i in range(1, n + 1):
        p = probabilities[i - 1]
        for j in range(i + 1):
            if j == 0:
                dp[i][j] = dp[i - 1][j] * (1 - p)
            else:
                dp[i][j] = dp[i - 1][j] * (1 - p) + dp[i - 1][j - 1] * p

    # Sum probabilities of having more than half 1s
    result = sum(dp[n][j] for j in range(half, n + 1))
    return result


# 贪心算法选择符合条件的不相交子集
def find_subsets(probabilities, threshold, n):
    selected_subsets = []
    used_indices = set()

    def is_valid_subset(subset):
        return probability_majority_one([probabilities[i] for i in subset]) >= threshold

    while len(selected_subsets) < n:
        best_subset = None
        best_probability = -1

        for i in range(len(probabilities)):
            if i in used_indices:
                continue
            # 尝试将单个元素作为子集
            subset = [i]
            if is_valid_subset(subset):
                best_subset = subset
                break

            for j in range(len(probabilities)):
                if j not in used_indices and j != i:
                    subset.append(j)
                    if is_valid_subset(subset):
                        subset_probability = sum(probabilities[k] for k in subset)
                        if (
                            best_subset is None
                            or len(subset) < len(best_subset)
                            or (
                                len(subset) == len(best_subset)
                                and subset_probability > best_probability
                            )
                        ):
                            best_subset = subset[:]
                            best_probability = subset_probability
                    subset.pop()  # backtrack

        if best_subset is None:
            break

        selected_subsets.append(best_subset)
        used_indices.update(best_subset)

    return selected_subsets


# Example usage
# probabilities = generate_normal_random_numbers(10, 0.7, 0.1)
# print(np.average(probabilities))
# result = probability_majority_one(probabilities)
# print(f"Theoretical Probability: {result:.4f}")

# 示例使用
# probabilities = generate_normal_random_numbers(10, 0.9, 0.01)
# print(probabilities)

probabilities= [0.9,0.9,0.9,0.9,0.9]
threshold = 0.9
n = 1

print(probability_majority_one([0.8817719897915676, 0.9050788723181453, 0.8712601322107582]))

selected_subsets = find_subsets(probabilities, threshold, n)
for subset in selected_subsets:
    tmp = [probabilities[i] for i in subset]
    print(tmp, probability_majority_one(tmp))
    print(math.ceil(len(tmp) / 2))

0.9642263962354349
[0.9] 0.9
1
